In [14]:
import os

In [15]:
%pwd

'/home/tanush-reddy/workspace/Wine-Grade-Estimator'

In [16]:
os.chdir("/home/tanush-reddy/workspace/Wine-Grade-Estimator")

In [17]:
%pwd

'/home/tanush-reddy/workspace/Wine-Grade-Estimator'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [19]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        self.config_file_path = config_file_path
        self.params_file_path = params_file_path
        self.schema_file_path = schema_file_path

        self.config = read_yaml(self.config_file_path)
        self.params = read_yaml(self.params_file_path)
        self.schema = read_yaml(self.schema_file_path)

        create_directories([self.config.artifacts_root])
        
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path)
        )
        return data_transformation_config

In [21]:
import os
from src import logger
from src.exception import CustomException
import sys
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
       
    def train_test_split(self):
        try:
            df = pd.read_csv(self.config.data_path)
            train,test=train_test_split(df, test_size=0.2, random_state=42)
            train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
            test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)
            logger.info("Data split into train and test sets successfully.")
            logger.info(train.shape)
            logger.info(test.shape)
        except Exception as e:
            raise CustomException(e,sys) from e   
    

In [24]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation= DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    logger.error(f"An error occurred: {str(e)}")
    raise CustomException(e,sys) from e   

[2025-07-29 00:26:26,672] 28 WineGradeEstimator - INFO - YAML file config/config.yaml read successfully.
[2025-07-29 00:26:26,673] 28 WineGradeEstimator - INFO - YAML file params.yaml read successfully.
[2025-07-29 00:26:26,675] 28 WineGradeEstimator - INFO - YAML file schema.yaml read successfully.
[2025-07-29 00:26:26,676] 46 WineGradeEstimator - INFO - Directory created: artifacts
[2025-07-29 00:26:26,677] 46 WineGradeEstimator - INFO - Directory created: artifacts/data_transformation
[2025-07-29 00:26:26,710] 12 WineGradeEstimator - INFO - Data split into train and test sets successfully.
[2025-07-29 00:26:26,710] 13 WineGradeEstimator - INFO - (1279, 12)
[2025-07-29 00:26:26,710] 14 WineGradeEstimator - INFO - (320, 12)
